In [46]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
from sklearn.linear_model import LinearRegression
import json
from sklearn.preprocessing import StandardScaler
import tensorflow 
from sklearn import metrics
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow import keras

with open("../input/time-series-forecasting/ipt_12.json") as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [47]:
def convert_to_numpy(series):
    return np.array(series)

In [48]:
def load_data(data_path):
    """
    Loading of the dataset provided
    Edit the code below
    """
    with open(data_path,"r") as f:
        data = json.load(f)
    data = pd.DataFrame(data)
    data = data.drop('94')
    return data

In [49]:
import numpy as np
from sklearn.model_selection import train_test_split

train,test = train_test_split(df, test_size=0.2, random_state=42)

In [53]:
def shorten_series(data_series):
    series = np.array(data_series)
    small_series = []
    for i in range(0,len(series),8):
        small_series.append(series[i])
    return np.array(small_series)


In [54]:
def preprocess_data(data):
    """
    A standard nan removal to be added.
    Add more preprocessing steps if needed.
    """
    X = []
    for i in range(data.shape[0]):
        series = np.stack((data.iloc[i]['smcAC'],data.iloc[i]['smcDC'],data.iloc[i]['vib_table'],data.iloc[i]['vib_spindle'],data.iloc[i]['AE_table'],data.iloc[i]['AE_spindle']),axis=1)
        X.append(series.reshape(1125,6))
    X = np.array(X)
    y = data['VB'].fillna(data['VB'].mean())
    return X,y

In [55]:
def split_train_test(X, y):
    """
    Splitting the data into train, test, validation 
    """
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [56]:
def normalization(X_train, X_val, X_test):
    scaler = StandardScaler()
    shp = X_train.shape[1]
    X_train = np.reshape(X_train, (-1,shp))
    X_val = np.reshape(X_val, (-1,shp))
    X_test = np.reshape(X_test, (-1,shp))
    
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    return np.reshape(X_train, (-1,shp,1)), np.reshape(X_val, (-1,shp,1)), np.reshape(X_test, (-1,shp,1))

In [57]:
def timeseries_transform(data, head_size, num_heads, ff_dim, dropout=0):
    """
    Implement the timeseries transformer here
    """
    # Normalization and Attention
    x = data
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Dropout(dropout)(x)
    res = x + data

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=data.shape[-1], kernel_size=1)(x)
    return x + res

In [58]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = timeseries_transform(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [59]:
def model_training(X_train, y_train, X_val, y_val):
    """
    Train the data with the compatible model
    """
    
    input_shape = X_train.shape[1:]

    model = build_model(input_shape, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128], mlp_dropout=0.4, dropout=0.35)
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=10000, decay_rate=0.9)

    model.compile(
        loss="mse",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[tensorflow.keras.metrics.MeanSquaredError()],
    )
    
    model.summary()

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    model.fit(X_train, y_train, validation_data=(X_val,y_val),  epochs=200, batch_size=16, callbacks=callbacks)
    return model

In [60]:
def metric(y_act, y_pred):
    """
    Standard metrics and plotting should be same
    Metrics should be computed on validation data(unseen data)
    1. Balanced accuracy score
    2. Confusion matrix
    3. Per-class accuracy
    """
    
    cm = metrics.confusion_matrix(y_act, y_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_act, y_pred)
    
    return cm, balanced_accuracy

In [61]:
def validation(X_val, y_val, metric):
    """
    Comparing the results with provided Series Embedder
    Plot confusion matrices of self analysis and LSTM with balanced_accuracy
    
    """
    
    score = model.evaluate(X_val, y_val, verbose=1)
    
    return score

In [62]:
def evaluate(X_test, y_act, metric, model):
    y_pred = model.predict(X_test, verbose=1)
    cm, ba = metric(y_act, y_pred)
    
    return y_pred, cm, ba

In [63]:
path = "../input/time-series-forecasting/ipt_12.json"
data = load_data(path)
data = data.head()
data['smcAC'] = data['smcAC'].apply(lambda x:shorten_series(x))
data['smcDC'] = data['smcDC'].apply(lambda x:shorten_series(x))
data['vib_table'] = data['vib_table'].apply(lambda x:shorten_series(x))
data['vib_spindle'] = data['vib_spindle'].apply(lambda x:shorten_series(x))
data['AE_table'] = data['AE_table'].apply(lambda x:shorten_series(x))
data['AE_spindle'] = data['AE_spindle'].apply(lambda x:shorten_series(x))
X, y = preprocess_data(data)

X_train, X_val, X_test, y_train, y_val, y_test = split_train_test(X, y)
model_self=model_training(X_train, y_train, X_val, y_val)


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1125, 6)]    0                                            
__________________________________________________________________________________________________
multi_head_attention_16 (MultiH (None, 1125, 6)      27654       input_5[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
layer_normalization_32 (LayerNo (None, 1125, 6)      12          multi_head_attention_16[0][0]    
__________________________________________________________________________________________________
dropout_36 (Dropout)            (None, 1125, 6)      0           layer_normalization_32[0][0

In [70]:
model_self.predict(X_test)

array([[0.8393218]], dtype=float32)